# Aprendizaje automátco relacional

# 1 Lectura de datos y creación del DataFrame con las métricas

In [226]:
import pandas as pd
import networkx as nx
import numpy as np
np.random.seed(357823)
import random
random.seed(357823)

En primer lugar, realizamos la lectura de los vértices y aristas y creamos un grafo con estas.

In [227]:
edges=pd.read_csv('edges.csv')
vertices=pd.read_csv('nodes.csv')

In [228]:
edges.head()

,source,target
0,Baron Zemo,Blackout (Marcus Daniels)
1,Baron Zemo,Vermin (comics)
2,N'astirh,S'ym
3,N'astirh,Belasco (Marvel Comics)
4,N'astirh,Madelyne Pryor


In [229]:
vertices.head()

,group,id,size
0,1,Baron Zemo,2
1,1,N'astirh,3
2,0,Silver Sable,1
3,0,Hope Summers (comics),1
4,0,Magik (Illyana Rasputin),1


In [230]:
G=nx.from_pandas_edgelist(edges,source='source',target='target')

Obtenemos las métricas relacionales para cada nodo. Hemos decidido filtrar los vértices según el tamaño de su componente conexa, ya que quedan algunos vértices aislados en componentes conexas muy pequeñas, donde las métricas relacionales no tienen sentido, y como los vértices no disponen de propiedades adicionales quedan ejemplos con atributos idénticos que solo entorpecen el entrenamiento de los modelos. Tras experimentación el umbral que se ha decido escoger es de 10.

Variable para activar o desactivar el filtrado de vértices, aunque por nuestra experiencia es mejor llevar a cabo el filtrado, porque quedan ejemplos que entorpecen el entrenamiento.

In [231]:
filtrado=True

In [232]:
from collections import Counter
degree=nx.degree(G)
triangles=nx.triangles(G)
degree_centrality= nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G)
clustering = nx.clustering(G)
tam_comp = {}
for i, component in enumerate(nx.connected_components(G)):
    for node in component:
        tam_comp[node] = len(component)
vertices_filtrados = vertices[vertices['id'].map(lambda v: tam_comp[v] >= 10)] if filtrado  else vertices

Convertimos las métricas obtenidas a lista para usarlas para crear el dataframe de los datos, utilizamos los vértices leídos del dataset al iterar para garantizar que todas tengan el mismo orden y que no se mezclen métricas de diferentes vértices.

In [233]:
degree_dict= {d[0]:d[1] for d in degree}
degreeL = [degree[v["id"]]for _,v in vertices_filtrados.iterrows()]
trianglesL=[triangles[v['id']] for _,v in vertices_filtrados.iterrows()]
degree_centralityL = [degree_centrality[v['id']] for _,v in vertices_filtrados.iterrows()]
closeness_centralityL = [closeness_centrality[v['id']] for _,v in vertices_filtrados.iterrows()]
betweenness_centralityL = [betweenness_centrality[v['id']] for _,v in vertices_filtrados.iterrows()]
eigenvector_centralityL =[eigenvector_centrality[v['id']] for _,v in vertices_filtrados.iterrows()]
clusteringL = [clustering[v['id']] for _,v in vertices_filtrados.iterrows()]

Crear el dataframe con los datos.

In [234]:
df = pd.DataFrame({'triangle':trianglesL, "degree_centrality":degree_centralityL,
                  'closeness_centrality':closeness_centralityL, 'betweenness_centrality':betweenness_centralityL,
                    'eigenvector_centrality':eigenvector_centralityL,'clustering':clusteringL,'group':vertices_filtrados['group']})



In [235]:
df.head()

,triangle,degree_centrality,closeness_centrality,betweenness_centrality,eigenvector_centrality,clustering,group
3,0,0.002865,0.065194,0.000000,1.361871e-03,0.000000,0
6,0,0.002865,0.009551,0.000000,1.666953e-17,0.000000,1
9,0,0.017192,0.110388,0.176275,1.224383e-01,0.000000,1
14,0,0.002865,0.052361,0.000000,3.509777e-05,0.000000,2
16,1,0.008596,0.070706,0.011593,6.506930e-04,0.333333,1


In [236]:
atributos=df.iloc[:,0:-1]
objetivo=df.iloc[:,-1]

# 2 Entrenamiento y evalución de los distintos modelos

LLevamos a cabo la división del conjunto de ejemplos de ejemplos en entrenamiento y prueba. Escoegmos el 80% como conjunto de entrenamiento y el 20% como conjunto de prueba. Se usa el parámetro stratify para garantizar que la proporción de las clases sea la misma en el conjunto de entrenamiento y en el de prueba.

In [237]:
from sklearn.model_selection import train_test_split
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo,random_state=357823)

Se normalizan los datos para garantizar que todas las propiedades contribuyan de igual medida y evitar desbordamientos numéricos.

In [238]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

normalizador = ColumnTransformer(
    [
        ('normalizador', MinMaxScaler(), list(range(0, 6)))
    ]
)

# 2.1 Árbol de decisión CART

Selección de mejores hiperparámetros para el Árbol de Decisión CART utilizando GridSearch:

In [239]:
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
rejilla_de_hiperparámetros_arbol = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'arbol__max_depth': range(3, 11),
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'arbol__min_samples_split': range(2, 20)
}
kfold = StratifiedKFold(n_splits=10,random_state=357823,shuffle=True)
tuberia_arbol=Pipeline([('normalizador',normalizador),('arbol',DecisionTreeClassifier(random_state=357823))])
búsqueda_en_rejilla_arbol = GridSearchCV(tuberia_arbol,
                                   rejilla_de_hiperparámetros_arbol,
                                   scoring='accuracy',
                                   cv=kfold
                                   )
búsqueda_en_rejilla_arbol.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=357823, shuffle=True),
             estimator=Pipeline(steps=[('normalizador',
                                        ColumnTransformer(transformers=[('normalizador',
                                                                         MinMaxScaler(),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('arbol',
                                        DecisionTreeClassifier(random_state=357823))]),
             param_grid={'arbol__max_depth': range(3, 11),
                         'arbol__min_samples_split': range(2, 20)},
             scoring='accuracy')

En este caso, los mejores hiperparámetros obtenidos son una profundidad máxima de 9 y un mínimo de 9 ejemplos para poder realizar una división en el árbol.

In [240]:
búsqueda_en_rejilla_arbol.best_params_

{'arbol__max_depth': 9, 'arbol__min_samples_split': 9}

Tasa de aciertos obtenido por este modelo en el proceso de validación cruzada:

In [241]:
print(búsqueda_en_rejilla_arbol.best_score_)

0.7262500000000001


Evaluamos el modelo sobre el conjunto de prueba (las filas 0,1 y 2 representan las distintas clases).

In [242]:

from sklearn.metrics import accuracy_score,classification_report
modelo_arbol=búsqueda_en_rejilla_arbol.best_estimator_
pred=modelo_arbol.predict(atributos_prueba)
print(accuracy_score(objetivo_prueba,modelo_arbol.predict(atributos_prueba)))
print(classification_report(objetivo_prueba,modelo_arbol.predict(atributos_prueba)))


0.717948717948718
              precision    recall  f1-score   support

           0       0.67      0.77      0.71        13
           1       0.81      0.77      0.79        22
           2       0.33      0.25      0.29         4

    accuracy                           0.72        39
   macro avg       0.60      0.60      0.60        39
weighted avg       0.71      0.72      0.71        39



Evaluamos el modelo sobre el conjunto de entrenamiento.

In [243]:
print(classification_report(objetivo_entrenamiento,modelo_arbol.predict(atributos_entrenamiento)))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89        51
           1       0.91      0.97      0.94        86
           2       0.75      0.56      0.64        16

    accuracy                           0.90       153
   macro avg       0.85      0.80      0.82       153
weighted avg       0.89      0.90      0.89       153



<h1>2.2 KNN (K-Nearest Neighbors)<h1>

Este algoritmo realiza una predicción a partir del valor de sus k instancias más cercanas. En este caso, utilizaremos la distancia euclidea para medir la distacia entre las instancias.
Usaremos una validación cruzada estratificada con 10 particiones.

In [244]:
from sklearn.neighbors import KNeighborsClassifier
rejilla_de_hiperparámetros_knn = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'knn__metric': ["euclidean","manhattan"],
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'knn__n_neighbors': range(1, 11)
}
kfold = StratifiedKFold(n_splits=10,random_state=357823,shuffle=True)
tuberia_knn=Pipeline([('normalizador',normalizador),('knn',KNeighborsClassifier())])
búsqueda_en_rejilla_knn = GridSearchCV(tuberia_knn,
                                   rejilla_de_hiperparámetros_knn,
                                   scoring='accuracy',
                                   cv=kfold)
búsqueda_en_rejilla_knn.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=357823, shuffle=True),
             estimator=Pipeline(steps=[('normalizador',
                                        ColumnTransformer(transformers=[('normalizador',
                                                                         MinMaxScaler(),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__metric': ['euclidean', 'manhattan'],
                         'knn__n_neighbors': range(1, 11)},
             scoring='accuracy')

In [245]:
búsqueda_en_rejilla_knn.best_score_

np.float64(0.6545833333333333)

Ahora vamos comprobar los resultados, para ello buscamos el mejor modelo encontrado en la busqueda de hiperparametros. Para ver los resultados utilizaremos la función classification_report() que nos muestra un reporte detallado con los datos que nos interesan, en esta celda lo haremos con el conjunto de prueba.

In [246]:
modelo_knn=búsqueda_en_rejilla_knn.best_estimator_
print(accuracy_score(objetivo_prueba,modelo_knn.predict(atributos_prueba)))
print(classification_report(objetivo_prueba,modelo_knn.predict(atributos_prueba)))

0.5897435897435898
              precision    recall  f1-score   support

           0       0.50      0.54      0.52        13
           1       0.78      0.64      0.70        22
           2       0.29      0.50      0.36         4

    accuracy                           0.59        39
   macro avg       0.52      0.56      0.53        39
weighted avg       0.63      0.59      0.61        39



En esta celda imprimiremos el reporte del conjunto de entrenamiento. 

In [247]:
print(classification_report(objetivo_entrenamiento,modelo_knn.predict(atributos_entrenamiento)))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        51
           1       1.00      0.99      0.99        86
           2       0.89      1.00      0.94        16

    accuracy                           0.99       153
   macro avg       0.96      0.99      0.98       153
weighted avg       0.99      0.99      0.99       153



# 2.3 Redes neuronales

In [248]:
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Normalization,Dense
from keras import Sequential, Input
from sklearn.utils.class_weight import compute_class_weight
from keras.optimizers import Adam,SGD
import tensorflow as tf
import os
tf.random.set_seed(357823)
os.environ['PYTHONHASHSEED'] = '357823'
os.environ['TF_DETERMINISTIC_OPS'] = '1'

Construimos la red neuronal con una capa de entrada de 6 variables. Añadimos como capas ocultas una de 50 neuronas con función de activación ReLu y otra de 10 neuronas con función de activación tangente hiperbólica. Para la variable objetivo se ha utilizado una codificación one-hot y se ha utilizado la función softmax como función de activación. Además, se ha escogido como función de pérdida la entropía cruzada categórica y como optimizador Adam, que ha resultado dar un rendimiento bastante bueno.

In [257]:
enc=OneHotEncoder()
objetivo_entrenamiento_cod=enc.fit_transform(objetivo_entrenamiento.to_frame()).toarray()
red = Sequential()
normalizador_red=Normalization()
normalizador_red.adapt(atributos_entrenamiento.to_numpy())
red.add(Input(shape=(6,)))
red.add(normalizador_red)
red.add(Dense(10,activation="relu"))
red.add(Dense(3,activation="softmax"))
red.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy',
                   metrics=['accuracy'])
red.fit(atributos_entrenamiento,objetivo_entrenamiento_cod,epochs=250,batch_size=256)

Epoch 1/250
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 629ms/step - accuracy: 0.3333 - loss: 1.1605
Epoch 2/250
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.3464 - loss: 1.1252
Epoch 3/250
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.4118 - loss: 1.0941
Epoch 4/250
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.4379 - loss: 1.0671
Epoch 5/250
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5098 - loss: 1.0439
Epoch 6/250
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5098 - loss: 1.0240
Epoch 7/250
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5098 - loss: 1.0069
Epoch 8/250
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.5098 - loss: 0.9921
Epoch 9/250
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5294 - loss: 0.9792
Epoch 10/250
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5556 - loss: 0.9679
Epoch 11/250
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5556 - loss: 0.9577
Epoch 12/250
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5556 - l

Evaluamos el moodelo sobre el conjunto de prueba.

In [258]:
pred=np.argmax(red.predict(atributos_prueba),axis=1)
print(classification_report(objetivo_prueba,pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
              precision    recall  f1-score   support

           0       0.53      0.77      0.62        13
           1       0.79      0.68      0.73        22
           2       1.00      0.25      0.40         4

    accuracy                           0.67        39
   macro avg       0.77      0.57      0.59        39
weighted avg       0.72      0.67      0.66        39



Evaluamos el modelo sobre el conjunto de entrenamiento.

In [259]:
pred_ent=np.argmax(red.predict(atributos_entrenamiento),axis=1)
print(classification_report(objetivo_entrenamiento,pred_ent))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
              precision    recall  f1-score   support

           0       0.66      0.61      0.63        51
           1       0.72      0.85      0.78        86
           2       0.60      0.19      0.29        16

    accuracy                           0.70       153
   macro avg       0.66      0.55      0.57       153
weighted avg       0.69      0.70      0.68       153



# 2.4 Naive Bayes

Discretizamos los valores de las distintas propiedades, ya que Naive Bayes no trabaja con atributos continuos

In [252]:
from sklearn.preprocessing import KBinsDiscretizer
discretizador = ColumnTransformer([('discretizador',
                                    KBinsDiscretizer(
    n_bins=5, 
    encode='ordinal',  
    strategy='uniform'  
),
                                   list(range(0,6)))])

In [253]:
from sklearn.naive_bayes import CategoricalNB
rejilla_de_hiperparámetros_naive_bayes = {
    'naive_bayes__alpha': range(1, 11),
    'discretizador__discretizador__n_bins':range(2,20)
}
kfold = StratifiedKFold(n_splits=5)
tuberia_naive_bayes=Pipeline([('discretizador',discretizador),('naive_bayes',CategoricalNB())])
búsqueda_en_rejilla_naive_bayes = GridSearchCV(tuberia_naive_bayes,
                                   rejilla_de_hiperparámetros_naive_bayes,
                                   scoring='accuracy',
                                   cv=kfold)
búsqueda_en_rejilla_naive_bayes.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('discretizador',
                                        ColumnTransformer(transformers=[('discretizador',
                                                                         KBinsDiscretizer(encode='ordinal',
                                                                                          strategy='uniform'),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('naive_bayes', CategoricalNB())]),
             param_grid={'discretizador__discretizador__n_bins': range(2, 20),
                         'naive_bayes__alpha': range(1, 11)},
             scoring='accuracy')

Rendimiento obtenido en el proceso de validación cruzada.

In [254]:
búsqueda_en_rejilla_naive_bayes.best_score_

np.float64(0.5819354838709677)

Evaluamos el modelo sobre el conjunto de prueba.

In [255]:
modelo_naive_bayes=búsqueda_en_rejilla_naive_bayes.best_estimator_
pred=modelo_naive_bayes.predict(atributos_prueba)
print(classification_report(objetivo_prueba,modelo_naive_bayes.predict(atributos_prueba)))

              precision    recall  f1-score   support

           0       0.25      0.08      0.12        13
           1       0.57      0.91      0.70        22
           2       0.00      0.00      0.00         4

    accuracy                           0.54        39
   macro avg       0.27      0.33      0.27        39
weighted avg       0.41      0.54      0.44        39



c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Evaluamos el modelo sobre el conjunto de entrenamiento.

In [256]:
print(classification_report(objetivo_entrenamiento,modelo_naive_bayes.predict(atributos_entrenamiento)))

              precision    recall  f1-score   support

           0       1.00      0.14      0.24        51
           1       0.59      1.00      0.74        86
           2       0.00      0.00      0.00        16

    accuracy                           0.61       153
   macro avg       0.53      0.38      0.33       153
weighted avg       0.66      0.61      0.50       153



c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo